To do:
- Convert reg_y into a (51,1) matrix
- Run a linear regression
    * Find important features

In [21]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

In [22]:
regression_short = pd.read_csv('../data/processed/regression_short.csv')

regression_short.head(2)

,state,state_FOODINSEC_10_12,state_FOODINSEC_13_15,state_LACCESS_POP15,state_LACCESS_LOWI15,state_MEDHHINC15,state_GROCPTH14
0,AL,17.9,17.6,7708.624532,2241.191144,59520.448276,0.729979
1,AK,12.1,13.3,14371.804328,5651.092540,39830.104478,0.185682


In [31]:
reg_X = regression_short.loc[:, ('state_FOODINSEC_10_12',
                                 'state_LACCESS_POP15',
                                 'state_LACCESS_LOWI15',
                                 'state_MEDHHINC15', 
                                 'state_GROCPTH14')]

reg_y = regression_short.loc[:, 'state_FOODINSEC_13_15']

In [36]:
scaler = MinMaxScaler()

scaler.fit(reg_X)
#scaler.fit(reg_y)

reg_X = scaler.transform(reg_X)
#reg_y = scaler.transform(reg_y)